![GWC](img/GWC_Logo_2020.png)

GWC (Girls Who Code) College Loops: [Officer Applications](https://urldefense.com/v3/__https://forms.gle/nXASsZ2huQPXbSR38__;!!Mih3wA!VCPf_NbZe3FHa9u_KpFgX74Evoo7uOEIycX5otpbZUkxGraHtSXCC4EIZYRctw$)


## Course Announcements

**Due Friday**:
- D4
- Q4
- A2
- Weekly Project Survey (*optional*)

**Notes**
- No Discussion Section today (Wed 1PM; Holly)
    - Note: David *does* have TA office hours today @ 11AM 

# Inference: Linear Regression

- **simple linear regression**
    - effect size
    - p-value
    - interpretation
- **multiple linear regression**
    - confounding
- working with **transformed data**

In [ ]:
import pandas as pd
import numpy as np

# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(style="white", font_scale=2)

# import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
# set plotting size parameter
plt.rcParams['figure.figsize'] = (17, 7)

# Statmodels & patsy
import patsy
import statsmodels.api as sm

import warnings
warnings.filterwarnings('ignore')

from scipy.stats import skewnorm

#improve resolution
#comment this line if erroring on your machine/screen
%config InlineBackend.figure_format ='retina'

## The Question

Does Poverty Percentage affect Teen Birth Rate?

$H_o$: There is no relationship between poverty percentage and teen birth rate ($\beta = 0$)

$H_a$: There is a relationship between poverty percentage and teen birth rate ($\beta \ne 0$)

## The Data

In [ ]:
# read in file; specify that it is tab-separated file
df = pd.read_csv('https://raw.githubusercontent.com/shanellis/datasets/master/index.txt', sep='\t')
df.head()

## Assumptions of Linear Regression

1. Linear relationship
2. No multicollinearity
3. No auto-correlation
4. Homoscedasticity

## Exploratory Plot

In [ ]:
sns.scatterplot(x='PovPct', y='TeenBrth', data=df, s=100)
plt.xlabel('Poverty Percentage')
plt.ylabel('Teen Birth Rate');

We see there is some sort of relationship here, but we want to *measure that realtionship*. Given that the increase appears linear over time (Assumption #1), linear regreasssion allows us to do that.

In [ ]:
plt.subplot(1, 2, 1)
sns.distplot(df['PovPct'])
plt.xlabel('Poverty Percentage')

plt.subplot(1, 2, 2)
sns.distplot(df['TeenBrth'])
plt.xlabel('Teen Birth Rate');

## Linear Regression

$$outcome = \beta_0 + \beta_1*predictor$$

First, let's be sure we're on the same page about what our outcome is. Here, we're intererested in whether **Poverty Percentage (predictor)** impacts **Teen Birth Rate (outcome)**.

In [ ]:
# We can specify our model matrix using `pastsy`.
outcome, predictors = patsy.dmatrices('TeenBrth ~ PovPct', df)
model = sm.OLS(outcome, predictors)

And, then we just have to fit the model and look at the results.

In [ ]:
## fit the model
results = model.fit()

## look at the results
print(results.summary())

### Clicker Question #1

What is the effect size of the relationship between Poverty Percentage and Teen Birth Rate?

- A) 15.67
- B) 2.03
- C) 4.032
- D) 0.495

There is a lot of information in there. Let's focus on the three pieces we discussed last lecture:

- `coef` : $\beta_1$ estimate explaining the effect size
- `std err` : standard error 
- `P>|t|` : the p-value

## Interpretation

$$outcome = \beta_0 + \beta_1*predictor$$

$$ Teen Birth = 15.67 + 2.03  * Poverty Percentage $$

If the Poverty Percentage were 0, the Teen Birth Rate would be **15.67** (The Intercept, $\beta_0$)

For every 1 unit increase in Poverty Percentage, you expect to see a **2.03** unit increase in Teen Birth Rate (The effect size, $\beta_1$)

In [ ]:
## look at the results
print(results.summary())

### Clicker Question #2

Which value represents the expected Teen Birth Rate if the Poverty Percentage were 0?

- A) 15.67
- B) 2.03
- C) 4.032
- D) 0.495

In [ ]:
# Plot the orginal data (as before)
sns.scatterplot(x='PovPct', y='TeenBrth', alpha=0.3, data=df, s=100)

# Generate and plot the model fit line
xs = np.arange(df['PovPct'].min(), df['PovPct'].max())
ys = 2.0255 * xs + 15.67
plt.plot(xs, ys, '--k', linewidth=4, label='Model')

plt.xlabel('Poverty Percentage')
plt.ylabel('Teen Birth Rate')
plt.legend();

The model (the line) mathematically describes the relationship between the data points, but it doesn't explain the relationship *perfectly*. (All models are wrong!)

Note that this line is drawn in the following way:
- if you were to draw a perpendicular line from each point to the line and calculate that distance
- if you were to sum the distance across all points
- this line is the one that minimizes that sum

### Clicker Question #3

If I told you the Poverty Percentage of a state was 15, what would you estimate would be its Teen Birth Rate? 
- A) ~ 0
- B) ~ 30
- C) ~ 40
- D) ~ 50
- E) ~ 60

In [ ]:
## look at the results
print(results.summary())

$$ Teen Birth = 15.67 + 2.03  * Poverty Percentage $$

### Clicker Question #4

Which of the following is a reasonable estimate (guess) for a state with a Poverty Percentage of 20?

- A) 55.99
- B) 56.27
- C) 56.5
- D) A - C
- E) None of the above

## Estimates

If I told you a locations' Poverty Percentage, what would you guess its Teen Birth Rate would be?

$$ Teen Birth = 15.67 + 2.03  * Poverty Percentage $$

In [ ]:
## if Poverty Percentage were 18
birth_rate = 15.67 + 2.03 * 18
birth_rate

In [ ]:
## if Poverty Percentage were 12
birth_rate = 15.67 + 2.03 * 12
birth_rate

### Clicker Question #5

What is our conclusion from this analysis? (Question: Does Poverty Percentage affect Teen Birth Rate?)

- A) Reject the null; There is no relationship between Poverty Percentage and Teen Birth Rate
- B) Reject the null; There is a relationship between Poverty Percentage and Teen Birth Rate
- C) Fail to reject the null; There is no relationship between Poverty Percentage and Teen Birth Rate
- D) Fail to reject the null; There is a relationship between Poverty Percentage and Teen Birth Rate


# Multiple Linear Regression

So far, we've only been looking at the relationship of one measure (Poverty Percentage) on Teen Birth. 

*But*, Poverty Percentage is likely *not* the only thing that affects Teen Birth Rate.

We could imagine that Violent Crime rates in a location may affect both Poverty Percentage and could possibly affect Teen Birth Rate. (A confounder!)

This is where **multiple linear regression** is incredibly helpful. Multiple linear regression allows you to measure the effect of multiple predictors on an outcome.

In [ ]:
outcome, predictors = patsy.dmatrices('TeenBrth ~ PovPct + ViolCrime ', df)
mod = sm.OLS(outcome, predictors)
res = mod.fit()
print(res.summary())

### Clicker Question #6

Which has a larger effect on Teen Birth Rate?

- A) Poverty Percentage
- B) Violent Crime
- C) Effect is equal across all predictors
- D) No predictors have an effect on outcome

$$ Teen Birth = 17.68 + (1.63  * Poverty Percentage) + (0.4 * Violent Crime) $$

If the Poverty Percntage *and* Violent Crime were both 0, the Teen Birth Rate would be **17.68** (The Intercept)

Holding Violent crime constant, for every 1 unit increase in Poverty Percentage, you expect to see a **1.63** unit increase in Teen Birth Rate (The effect size, $\beta_1$)

Holding Poverty Percentage constant, for every 1 unit increase in Violent crime, you'd expect to see a **0.4** unit increase in Teen Birth Rate (The effect size, $\beta_2$)

## No Multiple Colinearity

If we have two predictors, we'll want to consider how similar the values are between predictors (typically, before your generate the model, but we'll do it after for sake of argument today).

In [ ]:
# relationship between predictors
sns.scatterplot(x='PovPct', y='ViolCrime', data=df, s=100)
plt.xlabel('Poverty Percentage')
plt.ylabel('Violent Crime');

In [ ]:
df[df['ViolCrime'] > 50]

### Outlier handling

In [ ]:
# removing DC
df = df[df['ViolCrime'] < 50]

In [ ]:
# relationship between predictors
# after removing outlier
sns.scatterplot(x='PovPct', y='ViolCrime', data=df, s=100)
plt.xlabel('Poverty Percentage')
plt.ylabel('Violent Crime');

In [ ]:
# run model
outcome, predictors = patsy.dmatrices('TeenBrth ~ PovPct + ViolCrime', df)
mod = sm.OLS(outcome, predictors)  
res = mod.fit()
print(res.summary())

Holding Violent crime constant, for every 1 unit increase in Poverty Percentage, you expect to see a **1.19** unit increase in Teen Birth Rate (The effect size, $\beta_1$)

Holding Poverty Percentage constant, for every 1 unit increase in Violent crime, you'd expect to see a **1.63** unit increase in Teen Birth Rate (The effect size, $\beta_2$)

### Clicker Question #7

Which has a larger effect on Teen Birth Rate?

- A) Poverty Percentage
- B) Violent Crime
- C) Effect is equal across all predictors
- D) No predictors have an effect on outcome

Removing the DC outlier led to a different estimate for the effect of violent crime in these data. The inclusion or removal of data in your analyses can affect the results, so it's always important to carefully consider what question you're asking and 

# Regression with transformed data

So far, we've been working with data that were approximately Normal and didn't require transformation. But that won't always be the case...

In [ ]:
sleep = pd.read_csv('https://raw.githubusercontent.com/shanellis/datasets/master/msleep.csv')
sleep.head()

What if we were interested in quantifying the **effect that REM sleep has on total sleep**?

We may hypothesize that the more REM sleep an animal gets the less total sleep it needs.

In [ ]:
sns.distplot(sleep['sleep_rem'][sleep['sleep_rem'].notnull()],hist=True)
plt.xlabel('REM Sleep');

We saw previously that these data are skewed right, and discussed that transforming these data could help us use them in analysis...

In [ ]:
sns.scatterplot(x='sleep_rem', y='sleep_total', data=sleep, s=100)
plt.xlabel('REM Sleep')
plt.ylabel('Total Sleep');

This relationship is not linear...so linear regression would not be appropriate

### Log Transformation

In [ ]:
# add log-transofmed column to sleep_rem & brainwt
sleep['sleep_rem10'] = np.log10(sleep['sleep_rem'])

In [ ]:
sns.distplot(sleep['sleep_rem10'][sleep['sleep_rem'].notnull()], hist=True)
plt.xlabel('log10(REM Sleep)');

In [ ]:
sns.scatterplot(x='sleep_rem10', y='sleep_total', data=sleep, s=100)
plt.xlabel('log10(REM Sleep)')
plt.ylabel('Total Sleep');

### Linear regression with transformed data

In [ ]:
# carry out regression using log-transformed predictor
outcome, predictors = patsy.dmatrices('sleep_total ~ sleep_rem10', sleep)
mod = sm.OLS(outcome, predictors)
res = mod.fit()
print(res.summary())

Here, we see that the $\beta$ estimate for `sleep_rem10` is 10.89....but remember that this value is on the log scale.

$$ Total Sleep = 8.57 + 10.89  * log_{10}(REM Sleep) $$

To interpret this coefficient, we would say that

a 1 unit increase in REM sleep, Total Sleep increases by $\approx$ $10.89/100$ units.

In [ ]:
# interpreting a log transformed coefficient
10.89 / 100

So, for each 1 hour increase in REM Sleep, Total Sleep increases by 0.11 hours. 

That's not a large effect, but it is different than what we hypothesized at the beginning!

More on interpretation of log transformed variables can be read [here](https://www.cscu.cornell.edu/news/statnews/stnews83.pdf)

### Why is log-transformation not math trickery?

First, some reading on data transformation can be found [here](http://fmwww.bc.edu/repec/bocode/t/transint.html). It's not beautifully formatted, but I think it is written in a more-straightforward manner than some [other](https://stats.stackexchange.com/questions/4831/regression-transforming-variables/4833#4833) [places](https://stats.stackexchange.com/questions/298/in-linear-regression-when-is-it-appropriate-to-use-the-log-of-an-independent-va) [online](https://www.researchgate.net/post/Why_do_we_do_transformation_before_data_analysis).

The first thing to remember is there is (in most cases) nothing special about how the data are originally expressed.

In our example above, there is nothing about "hours" as the unit that was chosen that makes these data "correct".

So, while it _feels_ like data transformation is trickery, our initial unit of hours is...in some ways arbitrary and something that we chose.

This is where we'll start with our argument that it's ok to transform (or think of it as *re-express*) our data
so that it can be (still-accurately) used with well-studied models.

To make the point a little more concretely, pH (measurement of acidity) is measured on the log scale. It _could_ be measured (transformed) off of the log scale. Those measurements would still explain a compound's acidity...it would just be on a different scale.

In other words:
    
> "Transformations are needed because there is no guarantee that the world works on the scales it happens to be measured on."

What *does* differ however, is the interprtation. Linear scales tell us absolute change, while logarithmic scales tell us relative change.

In [ ]:
dat = skewnorm.rvs(5, 1, size=1000)
sns.distplot(dat, kde=False, bins=20);

In [ ]:
dat_log = np.log10(dat)
sns.distplot(dat_log, kde=False, bins=20);

In [ ]:
#original value
dat[0]

In [ ]:
# log 10 transformed value
dat_log[0]

In [ ]:
# math that is actually happening
10.0*dat_log[0]

In [ ]:
# linear scale tells you asolute 
# difference between two points
dat[1] - dat[0]

In [ ]:
# log scale tells you relative 
# difference between two points
dat_log[1] - dat_log[0]